In [ ]:
!pip install beautifulsoup4

In [2]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from tqdm.auto import tqdm

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
'Upgrade-Insecure-Requests': '1', 'Cookie': 'v2=1495343816.182.19.234.142', 'Accept-Encoding': 'gzip, deflate, sdch',
'Referer': "http://finviz.com/quote.ashx?t="}

def get_fundamental_data(df):
  for symbol in tqdm(df.index, desc="Processing stocks"):
    try:
      #url= ("http://finviz.com/quote.ashx?t=" + symbol. lower())
      r = requests.get("http://finviz.com/quote.ashx?t=" + symbol.lower(), headers=headers)
      soup = bs(r.content, 'html.parser')
      for m in df.columns:
        df.loc[symbol, m] = soup.find(text=m).find_next(class_='snapshot-td2').text
    except Exception as e:
        print(symbol, 'not found')
        print(e)
  return df

In [ ]:
stock_list = ['GOOG', 'TSLA', 'CSCO', 'JPM', 'KO', 'XOM', 'PLTR']

metric = [
    'P/B',
    'P/E',
    'Forward P/E',
    'PEG',
    'Debt/Eq',
    'EPS (ttm)',
    'EPS next 5Y',
    'Dividend %',
    'ROE',
    'ROI',
    'EPS Q/Q',
    'Insider Own'
]

In [ ]:
df = pd.DataFrame(index=stock_list, columns=metric)
df = get_fundamental_data(df)
df = df.replace('-', 0)
df

Processing stocks:   0%|          | 0/7 [00:00<?, ?it/s]

<ipython-input-12-97df192b90ab>:12: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  df.loc[symbol, m] = soup.find(text=m).find_next(class_='snapshot-td2').text


,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),EPS next 5Y,Dividend %,ROE,ROI,EPS Q/Q,Insider Own
GOOG,6.36,26.58,20.72,1.36,0.11,5.22,19.48%,0,25.33%,22.28%,46.01%,57.44%
TSLA,14.67,79.45,63.85,21.89,0.15,3.11,3.63%,0,23.13%,18.27%,-44.17%,13.03%
CSCO,4.29,14.45,11.85,2.25,0.19,3.31,6.41%,3.31%,31.78%,25.81%,37.21%,0.23%
JPM,1.53,9.17,9.97,3.06,2.24,16.75,3.00%,2.67%,16.83%,7.40%,38.62%,0.97%
KO,9.62,23.62,20.94,3.92,1.53,2.48,6.03%,3.09%,43.85%,17.80%,9.45%,0.63%
XOM,2.06,10.33,10.63,0.23,0.21,10.06,45.30%,3.56%,21.32%,17.41%,-52.03%,0.14%
PLTR,13.44,304.64,66.90,0,0.07,0.06,0,0,5.28%,4.37%,151.51%,16.16%


In [ ]:
df['EPS next 5Y'] = df['EPS next 5Y'].str.replace('%', '')
df['ROE'] = df['ROE'].str.replace('%', '')
df['ROI'] = df['ROI'].str.replace('%', '')
df['EPS Q/Q'] = df['EPS Q/Q'].str.replace('%', '')
df['Insider Own'] = df['Insider Own'].str.replace('%', '')
df = df.apply(pd.to_numeric, errors = 'coerce')
df

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),EPS next 5Y,Dividend %,ROE,ROI,EPS Q/Q,Insider Own
GOOG,6.36,26.58,20.72,1.36,0.11,5.22,19.48,0.0,25.33,22.28,46.01,57.44
TSLA,14.67,79.45,63.85,21.89,0.15,3.11,3.63,0.0,23.13,18.27,-44.17,13.03
CSCO,4.29,14.45,11.85,2.25,0.19,3.31,6.41,NaN,31.78,25.81,37.21,0.23
JPM,1.53,9.17,9.97,3.06,2.24,16.75,3.00,NaN,16.83,7.40,38.62,0.97
KO,9.62,23.62,20.94,3.92,1.53,2.48,6.03,NaN,43.85,17.80,9.45,0.63
XOM,2.06,10.33,10.63,0.23,0.21,10.06,45.30,NaN,21.32,17.41,-52.03,0.14
PLTR,13.44,304.64,66.90,0.00,0.07,0.06,NaN,0.0,5.28,4.37,151.51,16.16


In [ ]:
# Stocks that are quoted at low valuations
valuedf = df[(df['P/E'].astype(float) < 30) & (df['P/B'].astype(float) < 3)]
valuedf

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),EPS next 5Y,Dividend %,ROE,ROI,EPS Q/Q,Insider Own
JPM,1.53,9.17,9.97,3.06,2.24,16.75,3.0,NaN,16.83,7.40,38.62,0.97
XOM,2.06,10.33,10.63,0.23,0.21,10.06,45.3,NaN,21.32,17.41,-52.03,0.14


In [ ]:
# Stocks that have demonstrated earning power
EPSdf = df[df['EPS Q/Q'].astype(float) > 30]
EPSdf

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),EPS next 5Y,Dividend %,ROE,ROI,EPS Q/Q,Insider Own
GOOG,6.36,26.58,20.72,1.36,0.11,5.22,19.48,0.0,25.33,22.28,46.01,57.44
CSCO,4.29,14.45,11.85,2.25,0.19,3.31,6.41,NaN,31.78,25.81,37.21,0.23
JPM,1.53,9.17,9.97,3.06,2.24,16.75,3.00,NaN,16.83,7.40,38.62,0.97
PLTR,13.44,304.64,66.90,0.00,0.07,0.06,NaN,0.0,5.28,4.37,151.51,16.16


In [ ]:
# Stocks earning good returns on equity while employing little or no debt
equitydf = df[(df['Debt/Eq'].astype(float) < 1) & (df['ROE'].astype(float) > 10)]
equitydf

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),EPS next 5Y,Dividend %,ROE,ROI,EPS Q/Q,Insider Own
GOOG,6.36,26.58,20.72,1.36,0.11,5.22,19.48,0.0,25.33,22.28,46.01,57.44
TSLA,14.67,79.45,63.85,21.89,0.15,3.11,3.63,0.0,23.13,18.27,-44.17,13.03
CSCO,4.29,14.45,11.85,2.25,0.19,3.31,6.41,NaN,31.78,25.81,37.21,0.23
XOM,2.06,10.33,10.63,0.23,0.21,10.06,45.30,NaN,21.32,17.41,-52.03,0.14


In [ ]:
# Management having substantial ownership in the business
insiderdf = df[df['Insider Own'].astype(float) > .5]
insiderdf

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),EPS next 5Y,Dividend %,ROE,ROI,EPS Q/Q,Insider Own
GOOG,6.36,26.58,20.72,1.36,0.11,5.22,19.48,0.0,25.33,22.28,46.01,57.44
TSLA,14.67,79.45,63.85,21.89,0.15,3.11,3.63,0.0,23.13,18.27,-44.17,13.03
JPM,1.53,9.17,9.97,3.06,2.24,16.75,3.00,NaN,16.83,7.40,38.62,0.97
KO,9.62,23.62,20.94,3.92,1.53,2.48,6.03,NaN,43.85,17.80,9.45,0.63
PLTR,13.44,304.64,66.90,0.00,0.07,0.06,NaN,0.0,5.28,4.37,151.51,16.16
